# Классификация комментариев

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах: клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Описание данных**

Данные находятся в файле `/datasets/toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

**План работы**
1. Загрузим и подготовим данные.
2. Обучим разные модели.
3. Сделаем выводы.

In [24]:
import pandas as pd
import numpy as np
import string
import optuna
import plotly.express as px

from collections import defaultdict
from IPython.display import display

from fast_ml import eda
from ydata_profiling import ProfileReport

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score

from sklearn.metrics import (
    f1_score, auc, roc_curve, roc_auc_score, confusion_matrix
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [6]:
try:
    raw_comments = pd.read_csv('toxic_comments.csv')
except:
    raw_comments = pd.read_csv('/datasets/toxic_comments.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [7]:
display(eda.df_info(raw_comments))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
Unnamed: 0,int64,Numerical,159292,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",0,0.0
text,object,Categorical,159292,[Explanation\nWhy the edits made under my user...,0,0.0
toxic,int64,Numerical,2,"[0, 1]",0,0.0


Числовые распределения:

In [8]:
display(round(raw_comments.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,159292.0,79725.7,46028.84,0.0,39872.75,79721.5,119573.25,159450.0
toxic,159292.0,0.1,0.30,0.0,0.00,0.0,0.00,1.0


И детальный отчет:

In [9]:
ProfileReport(raw_comments).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы

# Подготовка данных для ML

Данные уже достаточно чистые, поэтому обработка будет относительно быстрой:

1. Отбросим первую колонку - она нам не нужна.
2. Причешем текст: приведем его к нижнему регистру, уберем пунктуацию, уберем стоп-слова, лемматизируем слова.

Создадим несколько функций, для обработки текста.

In [10]:
def lowercase_text(text):
    """
    Convert the provided text to lowercase.
    
    Parameters:
    - text (str): The text to be converted.
    
    Returns:
    - str: The text in lowercase.
    """
    return text.lower()

def remove_punctuation(text):
    """
    Remove punctuation from the provided text.
    
    Parameters:
    - text (str): The text from which punctuation should be removed.
    
    Returns:
    - str: The text without punctuation.
    """
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    """
    Remove stopwords from the provided text using NLTK's English stopwords list.
    
    Parameters:
    - text (str): The text from which stopwords should be removed.
    
    Returns:
    - str: The text without stopwords.
    """
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    """
    Lemmatize the provided text using NLTK's WordNetLemmatizer.
    
    Parameters:
    - text (str): The text to be lemmatized.
    
    Returns:
    - str: The lemmatized text.
    """
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

Применим эти функции.

In [11]:
df_comments = (
    raw_comments
    .drop('Unnamed: 0', axis=1)
    .assign(text=lambda x: x.text.apply(lowercase_text))
    .assign(text=lambda x: x.text.apply(remove_punctuation))
    .assign(text=lambda x: x.text.apply(remove_stopwords))
    .assign(text=lambda x: x.text.apply(lemmatize_text))
)

display(df_comments.head())

,text,toxic
0,explanation edits made username hardcore metal...,0
1,daww match background colour im seemingly stuc...,0
2,hey man im really trying edit war guy constant...,0
3,cant make real suggestion improvement wondered...,0
4,sir hero chance remember page thats,0


И теперь проверим word-cloud.

In [12]:
ProfileReport(df_comments).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы

## Модели ML

Создадим и обучим несколько моделей. Для начала разделим данные на `train` и `test` выборки. Создадим функцию для этого, чтобы сразу записать в датафрейм.

In [13]:
def split_data(df: pd.DataFrame, target_column: str, test_size: float, shuffle=False, stratify=None, split_ftr_tgt=True):
    """
    Split a DataFrame into training and testing datasets.

    This function accepts a DataFrame, the name of the target column, and the proportion of the data 
    to be included in the test split. Depending on the 'split_ftr_tgt' parameter, it either returns 
    four DataFrames (training features, training target, testing features, testing target) or two DataFrames 
    (complete training data, complete testing data).

    Args:
    - df (pd.DataFrame):
        The DataFrame to split. This DataFrame should include both the features and the target.

    - target_column (str): 
        The name of the target column. If 'split_ftr_tgt' is True, this column will be separated 
        from the features and returned in the target DataFrames.

    - test_size (float):
        The proportion of the data to include in the test split. For example, if `test_size` is 0.3, 
        30% of the data will be used for the test split, and the rest will be used for the training split.
        
    - shuffle (boolean):
        A flag to shuffle (True) or not (False) the data when splitting into train and test.

    - stratify (array-like):
        Determines the stratification strategy. If not None, data is split in a stratified fashion, using 
        the provided array as the class labels. It ensures that the training and test datasets have 
        approximately the same percentage of samples of each target class as the complete set.

    - split_ftr_tgt (boolean):
        If True, the function will return separate DataFrames for features and target for both training 
        and testing data. If False, it will return complete DataFrames for training and testing data.

    Returns:
    - list of pd.DataFrame:
        Depending on 'split_ftr_tgt':
            If True: A list containing four DataFrames - the training features, the training target, 
                     the testing features, and the testing target.
            If False: A list containing two DataFrames - the complete training data and the complete testing data.
    """
    df_train, df_test = train_test_split(
        df, test_size=test_size, random_state=RANDOM_SEED, shuffle=shuffle, stratify=stratify
    )
    
    if split_ftr_tgt:
        ftr_train = df_train.drop(target_column, axis=1)
        tgt_train = df_train[[target_column]]
        ftr_test = df_test.drop(target_column, axis=1)
        tgt_test = df_test[[target_column]]
        return [ftr_train, tgt_train, ftr_test, tgt_test]
    else:
        return [df_train, df_test]

У нас есть сильный дисбаланс классов в целевой переменной. Попробуем его решить с помощью `upsampling`.

In [14]:
def upsample_data(df, target_column):
    """
    Upsamples the minority class in the dataframe.
    
    Parameters:
    - df: DataFrame containing the data
    - target_column: str, name of the target column
    
    Returns:
    - DataFrame with upsampled minority class
    """
    # Separate the majority and minority classes
    df_majority = df[df[target_column] == 0]
    df_minority = df[df[target_column] == 1]

    # Upsample the minority class
    df_minority_upsampled = resample(
        df_minority, 
        replace=True,                              
        n_samples=len(df_majority),                
        random_state=RANDOM_SEED
    )

    # Combine the majority class with the upsampled minority class
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    # Shuffle the dataset to mix the two classes
    df_upsampled = df_upsampled.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    
    return df_upsampled

Сохраним выборки, проверим `upsampling`.

In [15]:
def print_class_balance(df, label):
    distribution = df.toxic.value_counts(normalize=True)
    print(f"Class balance {label}: {distribution[1]:.2f} - 1 and {distribution[0]:.2f} - 0")

# Define splits
dct_splits = {
    'train': split_data(df_comments, 'toxic', 0.1, split_ftr_tgt=False)[0],
    'test': split_data(df_comments, 'toxic', 0.1, split_ftr_tgt=False)[1]
}

# Check train-test split
print('Test to full sample size:', round(100 * dct_splits['test'].shape[0] / df_comments.shape[0], 2), '%')

# Upsample and show the class balance
print_class_balance(dct_splits['train'], 'before upsampling')
dct_splits['train'] = upsample_data(dct_splits['train'], 'toxic')
print_class_balance(dct_splits['train'], 'after upsampling')

# Update splits with separated features and target directly
dct_splits = {
    'train': {
        'features': dct_splits['train'].drop('toxic', axis=1),
        'target': dct_splits['train'][['toxic']]
    },
    'test': {
        'features': dct_splits['test'].drop('toxic', axis=1),
        'target': dct_splits['test'][['toxic']]
    }
}

Test to full sample size: 10.0 %
Class balance before upsampling: 0.10 - 1 and 0.90 - 0
Class balance after upsampling: 0.50 - 1 and 0.50 - 0


Выборки разбились как надо. `upsampling` удался.

Теперь проведем токенизацию и воспользуемся `tf-idf` для создания признаков.

In [16]:
 # Consider unigrams and bigrams, limit features to 5000 for computational efficiency
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

dct_splits['train']['features'] = vectorizer.fit_transform(dct_splits['train']['features']['text'])
dct_splits['test']['features'] = vectorizer.transform(dct_splits['test']['features']['text'])

dct_splits['train']['target'] = dct_splits['train']['target']['toxic']
dct_splits['test']['target'] = dct_splits['test']['target']['toxic']

Зададим `study` для `optuna` - она сделает для нас оптимальные модели.

In [17]:
def optimize_classifiers(ftr_train, tgt_train, n_trials: int):
    """
    Trains and optimizes classifiers using Optuna.

    Args:
    - ftr_train, tgt_train: Training features and target.
    - n_trials (int): The number of trials for Optuna optimization.

    Returns:
    - An Optuna study object containing the optimal model and its parameters.
    """
    # Ensure target is 1-d vector
    tgt_train = tgt_train.values.ravel()

    def get_classifier(trial):
        """
        Generate a classifier based on the suggestions from the Optuna trial.

        Parameters:
        - trial (optuna.Trial): 
            A trial is a process of evaluating an objective function. This object 
            provides interfaces to suggest hyperparameters.

        Returns:
        - classifier (object): 
            A classifier initialized with suggested hyperparameters.
        """
        classifiers = {
            'LogisticRegression': LogisticRegression(
                C=trial.suggest_float('lr_C', 0.001, 1, log=True),
                max_iter=trial.suggest_int('lr_max_iter', 100, 1000, step=100),
                random_state=RANDOM_SEED
            ),
            'DecisionTree': DecisionTreeClassifier(
                max_depth=trial.suggest_int('dt_max_depth', 1, 100),
                random_state=RANDOM_SEED
            ),
            'LightGBM': LGBMClassifier(
                max_depth=trial.suggest_int('lgbm_max_depth', 10, 20),
                n_estimators=trial.suggest_int('lgbm_n_estimators', 900, 1100),
                learning_rate=trial.suggest_float('lgbm_learning_rate', 0.2, 0.3),
                random_state=RANDOM_SEED
            ),
            'CatBoost': CatBoostClassifier(
                iterations=trial.suggest_int('cb_iterations', 400, 1000),
                learning_rate=trial.suggest_float('cb_learning_rate', 0.1, 0.4),
                logging_level='Silent',
                random_state=RANDOM_SEED
            ),
        }
        classifier_name = trial.suggest_categorical('classifier', list(classifiers.keys()))
        return classifiers[classifier_name]

    def objective(trial):
        """
        Objective function for Optuna optimization. Computes the F1-score for a given classifier.

        Args:
        - trial (optuna.Trial): 
            A trial is a process of evaluating an objective function. 
            This object is passed to an objective function and provides interfaces to 
            suggest hyperparameters.

        Returns:
        - float:
            Average F1-score from cross-validation of the classifier's predictions.
        """
        classifier_obj = get_classifier(trial)
        scores = cross_val_score(classifier_obj, ftr_train, tgt_train, scoring='f1', cv=3)
        
        return np.mean(scores)

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Возьмем результаты `study`.

In [18]:
study = optimize_classifiers(
    dct_splits['train']['features'],
    dct_splits['train']['target'],
    n_trials=10
)

Посмотрим, на параметры лучшей модели.

In [19]:
best_params = study.best_params
classifier_name = best_params['classifier']

param_prefixes = {
    'LogisticRegression': 'lr_',
    'DecisionTree': 'dt_',
    'LightGBM': 'lgbm_',
    'CatBoost': 'cb_'
}

# Filter the best_params to only include relevant parameters for the selected classifier
relevant_params = {k: v for k, v in best_params.items() if k.startswith(param_prefixes[classifier_name]) or k == 'classifier'}

formatted_params = "\n".join([f"  {key}: {value}" for key, value in relevant_params.items()])
print(f"Best params:\n{formatted_params}")

Best params:
  lgbm_max_depth: 17
  lgbm_n_estimators: 1078
  lgbm_learning_rate: 0.24722149251619494
  classifier: LightGBM


И историю подбора.

In [20]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.show()

fig = optuna.visualization.plot_slice(study, params=relevant_params)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.update_xaxes(tickangle=-90)
fig.show()

Процесс оптимизации показывает, что, учитывая имеющиеся данные и рассматриваемый диапазон гиперпараметров, классификатор `LightGBM` обеспечивает наименьшую ошибку `F1` на обучающих данных. Ожидаем, что эта модель покажет лучшие результаты на неизвестных данных среди всех рассмотренных моделей и гиперпараметров.

## Проверка результатов

Последним этапом, проверим, как модель работает на тестовых данных, а также, какие прогнозы у нас получились.

Для начала обучим лучшую модель.

In [21]:
model = LGBMClassifier(
    max_depth=study.best_params['lgbm_max_depth'],
    n_estimators=study.best_params['lgbm_n_estimators'],
    learning_rate=study.best_params['lgbm_learning_rate'],
    random_state=RANDOM_SEED
)

model.fit(dct_splits['train']['features'], dct_splits['train']['target']);

Посчитаем предсказания и посмотрим на метрики.

In [27]:
for sample in ['train', 'test']:
    dct_splits[sample]['prediction'] = model.predict(dct_splits[sample]['features'])
    print(
        'F1 score for', sample, 'dataset:', 
        round(f1_score(dct_splits[sample]['target'], dct_splits[sample]['prediction']), 3)
    )

F1 score for train dataset: 0.995
F1 score for test dataset: 0.766


## Выводы

На основе проведенного анализа и результатов моделирования можно сделать следующие выводы:

1. **Анализ данных:** Мы начали с 159,292 текстовых образцов с целью классифицировать, является ли каждый пост "токсичным". Данные не имели пропущенных значений, но были несбалансированы, с лишь 10% помеченных как токсичные.
  
2. **Преобразование данных:** Текстовые данные прошли комплексную предварительную обработку, что привело к формату, более подходящему для моделирования: мы убрали ненужные символы, привели текст к нижнему регистру, провели токенизацию и использовать `TF-IDF` для кодирования признаков. Чтобы противостоять несбалансированной целевой переменной, мы применили метод увеличения выборки для обучающих данных, достигнув равного соотношения между классами.

3. **Выбор и оптимизация модели:** Мы оценивали несколько классификаторов, включая логистическую регрессию, решающее дерево и ансамблевые модели, такие как `LightGBM` и `CatBoost`. Путем оптимизации гиперпараметров `LightGBM` показала наилудшие результаты.

4. **Метрики производительности:** Наша модель `LightGBM` показала F1 метрику в 0.995 на обучающих данных. Однако ее производительность на неизвестных тестовых данных была ниже с F1-баллом 0.766. Это указывает на переобучение модели: в качестве следующих шагов мы можем попробовать другие способы балансирования классов, более продвинутые способы кодирования признаков, а также другие модели типа `BERT`.